In [1]:
import numpy as np
from scipy import sparse
import pandas as pd

import re
import string
import time
import seaborn as sns
import itertools

In [40]:
import xgboost as xgb
#import lightgbm as lgb - will be used in following weeks
# from bayes_opt import BayesianOptimization - will be used in following weeks
import seaborn as sns
import matplotlib.pyplot as plt

import random
from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neural_network import MLPClassifier
from IPython.display import display
import datetime

%matplotlib inline 

/Users/jyguo/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [43]:
train_data = pd.read_csv('../input/air_visit_data.csv')
train_size = train_data.shape[0]

In [6]:
full_data = pd.read_csv('../input/full_data.csv')

/Users/jyguo/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,12,13,16,17,18,34,35,36,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature interactions

### numeric to numeric

In [8]:
full_data.head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,13,1.005227e-08,0
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,14,1.456087e-08,0
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,15,2.055908e-08,0
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,16,2.838881e-08,0
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,517,18,5.115756e-08,0


In [32]:
full_data[['latitude_air','mean_lat_air_lv1']]

,latitude_air,mean_lat_air_lv1
0,35.658068,35.675963
1,35.658068,35.675963
2,35.658068,35.675963
3,35.658068,35.675963
4,35.658068,35.675963
5,35.658068,35.675963
6,35.658068,35.675963
7,35.658068,35.675963
8,35.658068,35.675963
9,35.658068,35.675963


In [33]:
full_data['total_rvs'] = full_data['air_rvs'] + full_data['hpg_rvs']
full_data['mean_rvs_air_hpg'] = full_data[['air_rvs','hpg_rvs']].apply(lambda x:np.mean(x), axis=1)
full_data['mean_dt_diff_air_hpg'] = full_data[['air_rv_dt_diff','hpg_rv_dt_diff']].apply(lambda x:np.mean(x), axis=1)

# NEW FEATURES FROM Georgii Vyshnia ????
full_data['lon_plus_lat_air'] = full_data['longitude_air'] + full_data['latitude_air'] 

full_data['lat_to_mean_lat_air_lv1'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv1'])
full_data['lat_to_max_lat_air_lv1']  = full_data['latitude_air']-full_data['max_lat_air_lv1']
full_data['lat_to_min_lat_air_lv1']  = full_data['latitude_air']-full_data['min_lat_air_lv1']
full_data['lon_to_mean_lon_air_lv1']  = abs(full_data['longitude_air']-full_data['mean_lon_air_lv1'])
full_data['lon_to_max_lon_air_lv1']  = full_data['longitude_air']-full_data['max_lon_air_lv1']
full_data['lon_to_min_lon_air_lv1']  = full_data['longitude_air']-full_data['min_lon_air_lv1']
full_data['lat_to_mean_lat_air_lv2'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv2'])
full_data['lat_to_max_lat_air_lv2']  = full_data['latitude_air']-full_data['max_lat_air_lv2']
full_data['lat_to_min_lat_air_lv2']  = full_data['latitude_air']-full_data['min_lat_air_lv2']
full_data['lon_to_mean_lon_air_lv2'] = abs(full_data['longitude_air']-full_data['mean_lon_air_lv2'])
full_data['lon_to_max_lon_air_lv2']  = full_data['longitude_air']-full_data['max_lon_air_lv2']
full_data['lon_to_min_lon_air_lv2']  = full_data['longitude_air']-full_data['min_lon_air_lv2']

full_data['lat_to_mean_lat_hpg_lv1'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv1'])
full_data['lat_to_max_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv1']
full_data['lat_to_min_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv1']
full_data['lon_to_mean_lon_hpg_lv1']  = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv1'])
full_data['lon_to_max_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv1']
full_data['lon_to_min_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv1']
full_data['lat_to_mean_lat_hpg_lv2'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv2'])
full_data['lat_to_max_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv2']
full_data['lat_to_min_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv2']
full_data['lon_to_mean_lon_hpg_lv2'] = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv2'])
full_data['lon_to_max_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv2']
full_data['lon_to_min_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv2']

num_num_vars = ['total_rvs', 'mean_rvs_air_hpg',
       'mean_dt_diff_air_hpg', 'lon_plus_lat_air',
       'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1',
       'lat_to_min_lat_air_lv1', 'lon_to_mean_lon_air_lv1',
       'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
       'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2',
       'lat_to_min_lat_air_lv2', 'lon_to_mean_lon_air_lv2',
       'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
       'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1',
       'lat_to_min_lat_hpg_lv1', 'lon_to_mean_lon_hpg_lv1',
       'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
       'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2',
       'lat_to_min_lat_hpg_lv2', 'lon_to_mean_lon_hpg_lv2',
       'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2']

full_data.head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,lat_to_min_lat_hpg_lv1,lon_to_mean_lon_hpg_lv1,lon_to_max_lon_hpg_lv1,lon_to_min_lon_hpg_lv1,lat_to_mean_lat_hpg_lv2,lat_to_max_lat_hpg_lv2,lat_to_min_lat_hpg_lv2,lon_to_mean_lon_hpg_lv2,lon_to_max_lon_hpg_lv2,lon_to_min_lon_hpg_lv2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### categorical to categorical

In [38]:
full_data['air_area_genre'] = full_data['air_area_name'] + '-' + full_data['air_genre_name'] 
full_data['air_store_dow'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str) 
full_data['air_store_dow_holiday'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str) + '-' + full_data['holiday_flg'].astype(str)

cat_cat_vars = ['air_area_genre','air_store_dow', 'air_store_dow_holiday']
full_data[cat_cat_vars].head()

,air_area_genre,air_store_dow,air_store_dow_holiday
0,Tōkyō-to Minato-ku Shibakōen-Dining bar,air_ba937bf13d40fb24-2,air_ba937bf13d40fb24-2-0
1,Tōkyō-to Minato-ku Shibakōen-Dining bar,air_ba937bf13d40fb24-3,air_ba937bf13d40fb24-3-0
2,Tōkyō-to Minato-ku Shibakōen-Dining bar,air_ba937bf13d40fb24-4,air_ba937bf13d40fb24-4-0
3,Tōkyō-to Minato-ku Shibakōen-Dining bar,air_ba937bf13d40fb24-5,air_ba937bf13d40fb24-5-0
4,Tōkyō-to Minato-ku Shibakōen-Dining bar,air_ba937bf13d40fb24-0,air_ba937bf13d40fb24-0-0


In [41]:
start = time.time()

LBL = preprocessing.LabelEncoder()

cat_cat_LE_vars=[]
cat_cat_LE_map=dict()
for cat_var in cat_cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].astype(str))
    cat_cat_LE_vars.append(LE_var)
    cat_cat_LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (cat_cat_LE_vars))

end = time.time()
print(end - start)

Label Encoding air_area_genre
Label Encoding air_store_dow
Label Encoding air_store_dow_holiday
Label-encoded feaures: ['air_area_genre_le', 'air_store_dow_le', 'air_store_dow_holiday_le']
1.121849775314331


### Target aggregation
This sometimes works for timeseries problems.

In [44]:
tmp = full_data[:train_size].groupby(['air_store_id','dow','holiday_flg'])['visitors'].\
            agg([np.mean, np.max, np.min, np.median]).\
            reset_index().\
            rename(columns={'mean':'mean_visitors',
                           'amax':'max_visitors',
                           'amin':'min_visitors',
                           'median':'median_visitors'})
            
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])

tmp = full_data[:train_size].groupby(['air_store_id','dow', 'holiday_flg']).\
            apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )).\
            reset_index().rename(columns={0:'wmean_visitors'})
        
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])    
target_aggr_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']